In [1]:
import datacube
import xarray as xr
from joblib import load
import matplotlib.pyplot as plt
from datacube.utils.cog import write_cog
import pandas as pd
import geopandas as gpd
import numpy as np
import os
import json
import pickle

from deafrica_tools.datahandling import load_ard
from deafrica_tools.bandindices import calculate_indices
from deafrica_tools.dask import create_local_dask_cluster
from deafrica_tools.plotting import rgb, display_map
from deafrica_tools.classification import predict_xr
from deafrica_tools.spatial import xr_rasterize
    
from datacube.utils import geometry
from datacube.utils.cog import write_cog

from odc.io.cgroups import get_cpu_quota
from odc.algo import geomedian_with_mads, xr_geomedian

from feature_collection import feature_layers

/usr/local/lib/python3.8/dist-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.3-CAPI-1.16.1). Conversions between both will be slow.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/dask/dataframe/utils.py:367: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/usr/local/lib/python3.8/dist-packages/dask/dataframe/utils.py:367: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/usr/local/lib/python3.8/dist-packages/dask/dataframe/utils.py:367: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pa

## Create Dask cluster for running predictions

In [2]:
ncpus = round(get_cpu_quota())
print("ncpus = " + str(ncpus))

# client = create_local_dask_cluster(return_client=True, n_workers=1, threads_per_worker=ncpus)
create_local_dask_cluster()

ncpus = 62


Client Scheduler: tcp://127.0.0.1:38831 Dashboard: /user/whusggliuqx@gmail.com/proxy/8787/status,Cluster Workers: 1 Cores: 62 Memory: 512.40 GB


## Read in training data feaure names and class labels

In [3]:
# experiment_name = "exp_multipixel_allfeatures_removecorrfeaturesgt0p9_RandomForest"
experiment_name="rf_removecorrfeaturesgt0p9_simplified_cv"

In [4]:
# Get label dictionary
labels_path = "results/class_labels.json"
with open(labels_path, "r") as json_file:
    labels_dict = json.load(json_file)

# Get model features
feautres_path = f"results/{experiment_name}_features.json"
with open(feautres_path, "r") as json_file:
    features_dict = json.load(json_file)
    
features = features_dict["features"]

In [5]:
len(features)

68

## Load trained ML model and shapefile for prediction

To manage memory, we provide a shapefile that splits the area of interest into tiles, which are then looped over

In [6]:
# Choose model and load
model_path = f"results/{experiment_name}.joblib"
model = load(model_path).set_params(n_jobs=1)

output_crs="EPSG:32736"
# Choose file containing test areas and load
# AOIs_file = "data/Random_Squares_10km_within_AOI_Crop_subset.shp"
# AOIs_file = "data/AOI_Crop.shp"
# AOIs_file = "data/Crop_type_test_areas.shp"
AOIs_file = "data/Crop_type_test_areas_3_larger.shp"
AOIs_gdf = gpd.read_file(AOIs_file).to_crs(output_crs)

# Set results path
results_path = "results/"

In [7]:
AOIs_gdf

,id,geometry
0,None,"POLYGON ((520103.252 7850182.937, 520126.356 7..."
1,None,"POLYGON ((548162.280 7222518.913, 548232.887 7..."
2,None,"POLYGON ((1119729.592 8517666.380, 1120258.862..."


## Create the query for running the predictions
This uses the existing query from the training data collection notebook, and adds `dask_chunks` as an additional parameter.

In [8]:
# Load the query used for fitting
query_file = "results/query.pickle"

with open(query_file, "rb") as f:
    query = pickle.load(f)
    
# Specify any specific additions to the data query -- e.g. dask_chunks for enabling parallel computation
dask_chunks = {"x": 4000, "y": 4000}
query.update({"dask_chunks": dask_chunks})

query

{'annual_geomedian_times': {'annual_2021': '2021-01-01'},
 'semiannual_geomedian_times': {'semiannual_2021_07': '2021-07-01',
  'semiannual_2022_01': '2022-01-01'},
 'time_ranges': {'Q4_2021': slice('2021-10-01', '2021-12-31', None),
  'Q1_2022': slice('2022-01-01', '2022-03-31', None),
  'Q2_2022': slice('2022-04-01', '2022-06-30', None),
  'Q3_2022': slice('2022-07-01', '2022-09-30', None)},
 'resolution': (-10, 10),
 'output_crs': 'EPSG:32736',
 'dask_chunks': {'x': 4000, 'y': 4000}}

## Run model AOIs

The model will be run for each area of the shapefile, producing a prediction file and a probabilities file. These will be saved to the results folder.

If an area has already been processed, it will be skipped, and prediction will resume for any incomplete tiles. This is useful if the process fails partway through, or if you are logged out of the sandbox before completion

In [9]:
%%time
dc = datacube.Datacube(app="crop_type_ml")

# for index, aoi in AOIs_gdf.iterrows():
for index in range(0,len(AOIs_gdf)):
    aoi=AOIs_gdf.iloc[index]
    print(f"Processing Polygon {index}")
    
    # Check if polygon has already been processed. If so, skip
    output_filename = f"{results_path}/Test_area_{index}_croptype_prediction.tif"
#     if os.path.exists(output_filename):
#         print("Completed; Skipping")
#         continue

    # set up query based on aoi polygon
    geom = geometry.Geometry(geom=aoi.geometry, crs=AOIs_gdf.crs)
    query.update({"geopolygon": geom})

    # Load the feature data
    print("    Loading feature data")
    data = feature_layers(query)
#     data = feature_layers(query).persist()
    
    
    # Only keep features that are used by the model
    data = data[features]

#     data=data.fillna(0)
#     #predict using the imported model
#     predicted = predict_xr(model,
#                            data.unify_chunks(),
#                            proba=True,
#                            persist=True,
#                            clean=True,
#                            return_input=False
#                           ).astype(np.uint8).persist()
    predicted = predict_xr(model,
                           data,
                           proba=True,
                           persist=False,
                           clean=True,
                           return_input=False
                          ).compute().astype(np.uint8)
    # Load masks and clip
#     crop_mask_query = dict((k,query[k]) for k in ('resolution','output_crs','dask_chunks','geopolygon'))
    crop_mask_query = dict((k,query[k]) for k in ('resolution','output_crs','geopolygon'))
    crop_mask_query.update({"time": "2019"})

    # Load the crop mask
    print("    Loading crop_mask")
    crop_mask = dc.load(product="crop_mask", **crop_mask_query)
    
    # Create a mask for the aoi
    print("    Getting AOI mask")
    aoi_mask = xr_rasterize(
        gdf=gpd.GeoDataFrame({"Polygon": [index], "geometry": [aoi.geometry]}, crs=AOIs_gdf.crs),
        da=predicted,
        transform=predicted.geobox.transform,
        crs=output_crs,
    )

    # set the no data value
    NODATA = 255

    # Mask the predictions to
    print("    Preparing predictions")
    predicted_masked = (
#         predicted.Predictions.where((crop_mask.filtered == 1) & (aoi_mask==1), NODATA)
        predicted.Predictions.where((crop_mask.mask == 1) & (aoi_mask==1), NODATA) # revised: using unfiltered crop mask instead
    )
    
    predicted_masked.attrs["nodata"] = NODATA
    
    # Write to cog
    prediction_file = f"{results_path}/Test_area_{index}_croptype_prediction.tif"
    print(f"    Writing predictions to {prediction_file}")
    write_cog(
        predicted_masked,
        fname=prediction_file,
        overwrite=True,
        nodata=255,
    )
    
    del predicted_masked
    
    probability_masked = (
#         predicted.Probabilities.where((crop_mask.filtered == 1) & (aoi_mask==1), NODATA)
        predicted.Probabilities.where((crop_mask.mask == 1) & (aoi_mask==1), NODATA) # revised: using unfiltered crop mask instead
    )
    
    probability_masked.attrs["nodata"] = NODATA
    
    probabilities_file = f"{results_path}/Test_area_{index}_croptype_probabilities.tif"
    print(f"    Writing probabilities to {probabilities_file}")
    write_cog(
        probability_masked,
        fname=probabilities_file,
        overwrite=True,
        nodata=255,
    )
    
    del probability_masked
    
    del crop_mask
    del aoi_mask

    

Processing Polygon 0
    Loading feature data
predicting...


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


   probabilities...


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


    Loading crop_mask
    Getting AOI mask
    Preparing predictions
    Writing predictions to results//Test_area_0_croptype_prediction.tif
    Writing probabilities to results//Test_area_0_croptype_probabilities.tif
Processing Polygon 1
    Loading feature data
predicting...


...

    Loading crop_mask
    Getting AOI mask
    Preparing predictions
    Writing predictions to results//Test_area_1_croptype_prediction.tif
    Writing probabilities to results//Test_area_1_croptype_probabilities.tif
Processing Polygon 2
    Loading feature data
predicting...


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


   probabilities...


/usr/local/lib/python3.8/dist-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


    Loading crop_mask
    Getting AOI mask
    Preparing predictions
    Writing predictions to results//Test_area_2_croptype_prediction.tif
    Writing probabilities to results//Test_area_2_croptype_probabilities.tif
CPU times: user 20min 18s, sys: 1min 3s, total: 21min 22s
Wall time: 4h 11min 39s


In [14]:
! gdal_merge.py -o results/Test_areas_mosaic_croptype_merged_prediction.tif -co COMPRESS=Deflate -ot Byte results/Test_area_*_croptype_prediction.tif -init 255 -a_nodata 255
! gdal_merge.py -o results/Test_areas_mosaic_croptype_merged_probabilities.tif -co COMPRESS=Deflate -ot Byte results/Test_area_*_croptype_probabilities.tif -init 255 -a_nodata 255

0...10...20...30...40...50...60...70...80...90...100 - done.
0...10...20...30...40...50...60...70...80...90...100 - done.


distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
